# form

> this builds up an `.md` with a lot of nastiness to get to an HTML form to propose a new config file

In [ ]:
import re
from pathlib import Path

import importnb
import jinja2
from IPython.display import HTML

In [ ]:
URL = "https://github.com/deathbeds/jupyak/new/main"

In [ ]:
with importnb.Notebook():
    from jupyak.tasks import load_tasks
    from jupyak.tasks._yak import Repo, Yak

In [ ]:
def load_yak():
    load_tasks()
    return Yak({"pr": {}})

In [ ]:
def make_repo(repo: Repo):
    return []

In [ ]:
REPOS_TMPL = r"""
<h2>repos</h2>
defaults:<br/>
{% for name in yak.repos %}
    <input type="checkbox" name="show-repo-{{ name }}" id="show-repo-{{ name }}"/>
    <label class="show-repo-label" for="show-repo-{{ name }}">{{name}}</label>
{% endfor %}
<table style="position: relative;">
    <thead style="position: sticky; top: 0;">
        <tr>
            <th>repo</th>
            <th>baseline</th>
            <th>merge with</th>
        </tr>
    </thead>
    <tbody>
        {% for name, repo in yak.repos.items() %}
        {% set stem = "repos|" ~ name ~ "|github" %}
        {% set id_stem = "repos-" ~ name ~ "-github" %}
        {% set gh = repo.github %}
        {% set gh_pattern = "/(tree/[^s]+|pull/\d+|releases/tag/[^s]+)" %}
        <tr class="repo" id="repo-{{ name }}">
            <th>
                <code>{{ gh.url }}</code>
            </th>
            <td>
                <input id="{{ id_stem }}-baseline"
                    name="{{ stem }}|baseline"
                    type="text"
                    title="the baseline GitHub URL for {{ name }}"
                    spellcheck="false"
                    placeholder="a branch, tag, or PR (other than {{ gh.baseline.replace(gh.url, "") }})"
                    pattern="^$|^{{ gh_pattern }}"
                    data-jpyk-prefix="{{ gh.url }}"
                />
                <label for="{{ id_stem }}-baseline">
                    <br/>
                    must be empty, or one of
                    <code>/pull/{:a-number}</code>,
                    <code>/tree/{:a-branch}</code>,
                    <code>/releases/tag/{:a-tag}</code>.
                </label>
            </td>
            <td>
                <input id="{{ id_stem }}-merge_with"
                    name="{{ stem }}|merge_with"
                    title="one or more space-delimited GitHub URLs to merge into the {{ name }} baseline"
                    type="text"
                    spellcheck="false"
                    placeholder="one or more branch, tag, or PR"
                    data-jpyk-prefix="{{ gh.url }}"
                    pattern="^$|^{{ gh_pattern }}(\s+{{ gh_pattern}})*"
                />
                <label for="{{ id_stem }}-merge_with">
                    <br/>
                    must be empty, or one of
                    <code>/pull/{:a-number}</code>,
                    <code>/tree/{:a-branch}</code>,
                    <code>/releases/tag/{:a-tag}</code>.
                </label>
            </td>
        </tr>
        {% endfor %}
    </tbody>
</table>
"""

In [ ]:
def make_repos_form(yak: Yak):
    return [jinja2.Template(REPOS_TMPL).render(yak=yak)]

In [ ]:
REPO_STYLE_TEMPL = """
{% for name in yak.repos %}
#show-repo-{{ name }}:not(:checked) ~ table tbody #repo-{{ name }} {
    display: none;
}
{% endfor %}
"""

In [ ]:
def make_repos_style(yak: Yak):
    return [jinja2.Template(REPO_STYLE_TEMPL).render(yak=yak)]

In [ ]:
def make_form():
    yak = load_yak()
    chunks = []
    style_chunks = []
    script_chunks = []
    for trait_name, value in yak.trait_values().items():
        maker = globals().get(f"make_{trait_name}_form")
        if maker:
            chunks += maker(yak)
        styler = globals().get(f"make_{trait_name}_style")
        if styler:
            style_chunks += styler(yak)

    return [
        "<style>",
        *style_chunks,
        "</style>",
        """<form id="new">""",
        *chunks,
        "</form>",
        *script_chunks,
    ]

In [ ]:
if __name__ == "__main__":
    display(HTML("\n".join(make_form())))

In [ ]:
def write_form(dest: Path, **options):
    dest.parent.mkdir(parents=True, exist_ok=True)
    chunks = ["# new", *make_form()]
    txt = re.sub(r"^ +", "", "\n".join(chunks), flags=re.M)
    dest.write_text(txt, encoding="utf-8")